In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
wp_filetypes = 'https://en.wikipedia.org/wiki/List_of_file_formats'

In [3]:
page = requests.get(wp_filetypes)

In [4]:
soup = BeautifulSoup(page.text,'html.parser')

In [5]:
#idenfity the heading tags
#create dict to track starting line of each heading tag
heading_tags = ["h1", "h2", "h3","h4"]
heading_pos = [i.sourceline for i in soup.find_all(heading_tags)]
heading_text = [i.text for i in soup.find_all(heading_tags)]

#identify the content that we are interested in and the related links
#track the line positions of each

#text and position of content
content_pos = [i.sourceline for i in soup.find_all('li')]
content_text = [i.text for i in soup.find_all('li')]

#links
href_pos = [i.sourceline for i in soup.find_all('a',href=True)]
href_link = [i['href'] for i in soup.find_all('a',href=True)]


In [6]:
def get_tag_pos(poslist,contentlist):
    #construct a dict to hold the starting and ending line of each element you are processing
    #input: two lists of same length
    #output: dict of tuples with start and end pos
    poses = {}

    for item in range(len(contentlist)):
        begin = poslist[item]
        
        #create tuple with beginning and ending indexes of each heading
        if item == len(poslist)-1:
            end = content_pos[-1] #if end of heading list take last content tag pos
        else:
            end = poslist[item+1]-1  #next line minus 1
        
        poses[contentlist[item]] = (begin,end)

    return poses

In [7]:
heading_scope = get_tag_pos(heading_pos,heading_text)

In [8]:
content_scope = get_tag_pos(content_pos,content_text)

In [9]:
links_scope = get_tag_pos(href_pos,href_link)

In [10]:
def connect_links(content,links):
    #create dict to connect the text from list content 
    #to relevant tags
    #input: two dicts
    #output: nested dict
    content_links = {}

    for text in content:
        
        #nest dict holds the position of the content and the link
        temp = {}
        temp['pos'] = content[text]
        temp['link'] = []
                    
        for link in links:
            if content[text][0] <= links[link][0] <= content[text][1]:
                temp['link'].append(link)

        content_links[text] = temp
    
    return content_links

In [11]:
files_links = connect_links(content_scope,links_scope)

In [12]:
files_links

{'1 Archive and compressed\n\n1.1 Physical recordable media archiving\n1.2 Other Extensions\n\n': {'pos': (60,
   61),
  'link': ['#Archive_and_compressed']},
 '1.1 Physical recordable media archiving': {'pos': (62, 62),
  'link': ['#Physical_recordable_media_archiving']},
 '1.2 Other Extensions': {'pos': (63, 65), 'link': ['#Other_Extensions']},
 '2 Computer-aided design\n\n2.1 Computer-aided design (CAD)\n2.2 Electronic design automation (EDA)\n2.3 Test technology\n\n': {'pos': (66,
   67),
  'link': ['#Computer-aided_design']},
 '2.1 Computer-aided design (CAD)': {'pos': (68, 68),
  'link': ['#Computer-aided_design_(CAD)']},
 '2.2 Electronic design automation (EDA)': {'pos': (69, 69),
  'link': ['#Electronic_design_automation_(EDA)']},
 '2.3 Test technology': {'pos': (70, 72), 'link': ['#Test_technology']},
 '3 Database': {'pos': (73, 73), 'link': ['#Database']},
 '4 Big Data (Distributed)': {'pos': (74, 74),
  'link': ['#Big_Data_(Distributed)']},
 '5 Desktop publishing': {'pos': (

In [13]:
heading_scope

{'List of file formats': (41, 57),
 'Contents': (58, 198),
 'Archive and compressed[edit]': (199, 320),
 'Physical recordable media archiving[edit]': (321, 344),
 'Other Extensions[edit]': (345, 346),
 'Computer-aided design[edit]': (347, 349),
 'Computer-aided design (CAD)[edit]': (350, 454),
 'Electronic design automation (EDA)[edit]': (455, 489),
 'Test technology[edit]': (490, 493),
 'Database[edit]': (494, 558),
 'Big Data (Distributed)[edit]': (559, 562),
 'Desktop publishing[edit]': (563, 582),
 'Document[edit]': (583, 675),
 'Financial records[edit]': (676, 680),
 'Financial data transfer formats[edit]': (681, 685),
 'Font file[edit]': (686, 709),
 'Geographic information system[edit]': (710, 728),
 'Graphical information organizers[edit]': (729, 737),
 'Graphics[edit]': (738, 739),
 'Color palettes[edit]': (740, 744),
 'Color management[edit]': (745, 746),
 'Raster graphics[edit]': (747, 824),
 'Vector graphics[edit]': (825, 861),
 '3D graphics[edit]': (862, 954),
 'Links and 

In [18]:
def join_hdr_content(hdr,content):
    #join the headers with the content (file types and links) based on position

    hdr_cont_out_txt = dict(zip(hdr.keys(), ([] for _ in hdr.keys())))
    hdr_cont_out_lnk = dict(zip(hdr.keys(), ([] for _ in hdr.keys())))

    for h in hdr:
        for c in content:
            if hdr[h][0] <= c['pos'][0] <= hdr[h][1]:
                hdr_cont_out_txt[h].append(c)
                hdr_cont_out_lnk[h].append(content[c]['link'][0])


        
    return hdr_cont_out_txt, hdr_cont_out_lnk
    

In [19]:
join_hdr_content(heading_scope,files_links)

TypeError: string indices must be integers